In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!cp -avr /content/drive/MyDrive/Colab\ Notebooks/modified\ version/Checkpoint .
!cp -avr /content/drive/MyDrive/Colab\ Notebooks/modified\ version/samples .

'/content/drive/MyDrive/Colab Notebooks/modified version/Checkpoint' -> './Checkpoint'
'/content/drive/MyDrive/Colab Notebooks/modified version/Checkpoint/bestmodel2.pth' -> './Checkpoint/bestmodel2.pth'
'/content/drive/MyDrive/Colab Notebooks/modified version/Checkpoint/bestmodel1.pth' -> './Checkpoint/bestmodel1.pth'
'/content/drive/MyDrive/Colab Notebooks/modified version/Checkpoint/bestmodel.pth' -> './Checkpoint/bestmodel.pth'
'/content/drive/MyDrive/Colab Notebooks/modified version/samples' -> './samples'


In [8]:
!python '/content/drive/MyDrive/Colab Notebooks/modified version/generate.py'

Model loaded.
Beginning of Generation ....
Begin Generation of image .... 0
Generation of image .... 0
Begin Generation of image .... 1
Generation of image .... 1
Begin Generation of image .... 2
Generation of image .... 2
Begin Generation of image .... 3
Generation of image .... 3
Begin Generation of image .... 4
Generation of image .... 4
Begin Generation of image .... 5
Generation of image .... 5
Begin Generation of image .... 6
Generation of image .... 6
Begin Generation of image .... 7
Generation of image .... 7
Begin Generation of image .... 8
Generation of image .... 8
Begin Generation of image .... 9
Generation of image .... 9
Begin Generation of image .... 10
Generation of image .... 10
Begin Generation of image .... 11
Generation of image .... 11
Begin Generation of image .... 12
Generation of image .... 12
Begin Generation of image .... 13
Generation of image .... 13
Begin Generation of image .... 14
Generation of image .... 14
Begin Generation of image .... 15
Generation of

In [2]:
!cp -avr /content/drive/MyDrive/Colab\ Notebooks/modified\ version/sde.py .
!cp -avr /content/drive/MyDrive/Colab\ Notebooks/modified\ version/UNET.py .
!cp -avr /content/drive/MyDrive/Colab\ Notebooks/modified\ version/utils.py .
!cp -avr /content/drive/MyDrive/Colab\ Notebooks/modified\ version/diffusion_utils.py .

'/content/drive/MyDrive/Colab Notebooks/modified version/sde.py' -> './sde.py'
'/content/drive/MyDrive/Colab Notebooks/modified version/UNET.py' -> './UNET.py'
'/content/drive/MyDrive/Colab Notebooks/modified version/utils.py' -> './utils.py'
'/content/drive/MyDrive/Colab Notebooks/modified version/diffusion_utils.py' -> './diffusion_utils.py'


In [6]:
import torch 
import torchvision
import numpy as np
import os
from tqdm import tqdm, trange
import argparse


from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.nn import DataParallel
from torch.optim import lr_scheduler
from sde import VariancePreservingSDE, PluginReverseSDE
from UNET import UNet
input_channels = 1
input_height = 28
dimx = input_channels * input_height ** 2
real=True
debias=False
vtype= 'rademacher'
T0=1.0
model = UNet(
input_channels=input_channels,#channels
input_height=input_height, #image_size
ch=32,#dim
ch_mult=(1, 2, 2),
num_res_blocks=2,
attn_resolutions=(16,),
resamp_with_conv=True,
)
T = torch.nn.Parameter(torch.FloatTensor([T0]), requires_grad=False)

inf_sde = VariancePreservingSDE(beta_min=0.1, beta_max=20.0, T=T)
sde_model = PluginReverseSDE(inf_sde, model, T, vtype=vtype, debias=debias)

sde_model=torch.load(os.path.join('/content/drive/MyDrive/Colab Notebooks/modified version/Checkpoint', 'bestmodel.pth'))
sde_model.eval()
print('Model loaded.')
print("Beginning of Generation ....")
for j in range(1024):    
        with torch.no_grad():
                  print("Begin Generation of image ....",j)
                  mean=0
                  std=1
                  delta = sde_model.T / 50
                  y0 = torch.randn(1, input_channels, input_height, input_height).to(sde_model.T)
                  y0 = y0 * std + mean
                  ts = torch.linspace(0, 1, 50 + 1).to(y0) * sde_model.T
                  ones = torch.ones(1, 1, 1, 1).to(y0)
                  for i in range(50):
                        mu = sde_model.mu(ones * ts[i], y0)
                        sigma = sde_model.sigma(ones * ts[i], y0)
                        y0 = y0 + delta * mu + delta ** 0.5 * sigma * torch.randn_like(y0)
                  y0 = torch.clip(y0, 0, 1)
                  y0 = y0.view(1, 1, input_channels, input_height, input_height).permute(2, 0, 3, 1, 4).contiguous().view(input_channels, 1 * input_height, 1 * input_height)
                  
                  torchvision.utils.save_image(y0, os.path.join('/content/drive/MyDrive/Colab Notebooks/modified version/samples', f'{j}.png'))
                  print("Generation of image ....",j)
print("End of Generation ....")
     


Model loaded.
Beginning of Generation ....
Begin Generation of image .... 0
Generation of image .... 0
Begin Generation of image .... 1
Generation of image .... 1
Begin Generation of image .... 2
Generation of image .... 2
Begin Generation of image .... 3
Generation of image .... 3
Begin Generation of image .... 4
Generation of image .... 4
Begin Generation of image .... 5
Generation of image .... 5
Begin Generation of image .... 6
Generation of image .... 6
Begin Generation of image .... 7
Generation of image .... 7
Begin Generation of image .... 8
Generation of image .... 8
Begin Generation of image .... 9
Generation of image .... 9
Begin Generation of image .... 10
Generation of image .... 10
Begin Generation of image .... 11
Generation of image .... 11
Begin Generation of image .... 12
Generation of image .... 12
Begin Generation of image .... 13
Generation of image .... 13
Begin Generation of image .... 14
Generation of image .... 14
Begin Generation of image .... 15
Generation of

KeyboardInterrupt: ignored